<a href="https://colab.research.google.com/github/Vedmani/wear-particle-classification/blob/main/Xception.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install wandb -qU
!wandb login

     |████████████████████████████████| 1.9 MB 38.7 MB/s 
     |████████████████████████████████| 162 kB 68.5 MB/s 
     |████████████████████████████████| 182 kB 69.8 MB/s 
     |████████████████████████████████| 63 kB 1.3 MB/s 
     |████████████████████████████████| 162 kB 73.2 MB/s 
     |████████████████████████████████| 158 kB 64.3 MB/s 
     |████████████████████████████████| 157 kB 74.5 MB/s 
     |████████████████████████████████| 157 kB 77.7 MB/s 
     |████████████████████████████████| 157 kB 77.8 MB/s 
     |████████████████████████████████| 157 kB 79.7 MB/s 
     |████████████████████████████████| 157 kB 72.4 MB/s 
     |████████████████████████████████| 157 kB 72.3 MB/s 
     |████████████████████████████████| 157 kB 64.2 MB/s 
     |████████████████████████████████| 156 kB 68.7 MB/s 
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wand

In [3]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
import wandb
from wandb.keras import WandbCallback

In [4]:
tf.keras.utils.set_random_seed(123)
tf.config.experimental.enable_op_determinism()

In [5]:
PATH = '/content/drive/MyDrive/Research Internship/wear particles'
batch_size = 32
img_height = 224
img_width = 224

train_ds = tf.keras.utils.image_dataset_from_directory(
  PATH,
  validation_split=0.2,
  subset="training",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

val_ds = tf.keras.utils.image_dataset_from_directory(
  PATH,
  validation_split=0.2,
  subset="validation",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

class_names = train_ds.class_names
print(class_names)
num_classes = len(class_names)

Found 740 files belonging to 5 classes.
Using 592 files for training.
Found 740 files belonging to 5 classes.
Using 148 files for validation.
['cutting wear', 'normal rubbing and cutting wear combined', 'normal rubbing wear', 'red oxide', 'severe sliding wear']


In [6]:
xception = tf.keras.applications.Xception(
    include_top=False,
    weights="imagenet",
    input_tensor=None,
    input_shape=None,
    pooling=None)
data_augmentation = tf.keras.Sequential([layers.RandomFlip("horizontal_and_vertical"),layers.RandomRotation(0.2),])
model_xception=Sequential([
  #data_augmentation,
  layers.Rescaling(1./255, input_shape=(img_height, img_width, 3)),
  xception,
  layers.Flatten(),
  layers.Dense(32, activation='relu'),
  #layers.Dropout(0.5), 
  layers.Dense(num_classes, activation='softmax')
])
xception.trainable=False
model_xception.summary()

83683744/83683744 [==============================] - 0s 0us/step
Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 rescaling (Rescaling)       (None, 224, 224, 3)       0         
                                                                 
 xception (Functional)       (None, None, None, 2048)  20861480  
                                                                 
 flatten (Flatten)           (None, 100352)            0         
                                                                 
 dense (Dense)               (None, 32)                3211296   
                                                                 
 dense_1 (Dense)             (None, 5)                 165       
                                                                 
Total params: 24,072,941
Trainable params: 3,211,461
Non-trainable params: 20,861,480
___________________________________

In [7]:
model_xception.compile(
              optimizer= tf.keras.optimizers.Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-07, amsgrad=False, name='Adam'),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
              metrics=['accuracy']
              )

In [8]:
wandb.init(
      # Set the project where this run will be logged
      project="Transfer_learning", 
      name = "Xception",
      notes = "without augment and without dropout"
)

ERROR:wandb.jupyter:Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: vedmanivaidya. Use `wandb login --relogin` to force relogin


In [9]:
%%time
epochs=40
history = model_xception.fit(
  train_ds,
  validation_data=val_ds,
  epochs=epochs,
callbacks=[WandbCallback()]
)

wandb: WARNING The save_model argument by default saves the model in the HDF5 format that cannot save custom objects like subclassed models and custom layers. This behavior will be deprecated in a future release in favor of the SavedModel format. Meanwhile, the HDF5 model is saved as W&B files and the SavedModel as W&B Artifacts.


Epoch 1/40
19/19 [==============================] - ETA: 0s - loss: 0.5591 - accuracy: 0.8361

wandb: Adding directory to artifact (/content/wandb/run-20221018_055316-3ndxkssy/files/model-best)... Done. 1.0s


19/19 [==============================] - 196s 5s/step - loss: 0.5591 - accuracy: 0.8361 - val_loss: 0.2152 - val_accuracy: 0.9392
Epoch 2/40
19/19 [==============================] - ETA: 0s - loss: 0.0185 - accuracy: 0.9932

wandb: Adding directory to artifact (/content/wandb/run-20221018_055316-3ndxkssy/files/model-best)... Done. 1.0s


19/19 [==============================] - 22s 1s/step - loss: 0.0185 - accuracy: 0.9932 - val_loss: 0.1691 - val_accuracy: 0.9527
Epoch 3/40
19/19 [==============================] - 4s 192ms/step - loss: 0.0085 - accuracy: 0.9966 - val_loss: 0.1976 - val_accuracy: 0.9595
Epoch 4/40
19/19 [==============================] - ETA: 0s - loss: 0.0012 - accuracy: 1.0000

wandb: Adding directory to artifact (/content/wandb/run-20221018_055316-3ndxkssy/files/model-best)... Done. 1.0s


19/19 [==============================] - 22s 1s/step - loss: 0.0012 - accuracy: 1.0000 - val_loss: 0.1225 - val_accuracy: 0.9730
Epoch 5/40
19/19 [==============================] - ETA: 0s - loss: 4.9289e-04 - accuracy: 1.0000

wandb: Adding directory to artifact (/content/wandb/run-20221018_055316-3ndxkssy/files/model-best)... Done. 1.0s


19/19 [==============================] - 22s 1s/step - loss: 4.9289e-04 - accuracy: 1.0000 - val_loss: 0.1181 - val_accuracy: 0.9730
Epoch 6/40
19/19 [==============================] - 4s 197ms/step - loss: 3.7556e-04 - accuracy: 1.0000 - val_loss: 0.1236 - val_accuracy: 0.9730
Epoch 7/40
19/19 [==============================] - 4s 196ms/step - loss: 3.1766e-04 - accuracy: 1.0000 - val_loss: 0.1289 - val_accuracy: 0.9730
Epoch 8/40
19/19 [==============================] - 4s 197ms/step - loss: 2.7429e-04 - accuracy: 1.0000 - val_loss: 0.1321 - val_accuracy: 0.9730
Epoch 9/40
19/19 [==============================] - 4s 196ms/step - loss: 2.4905e-04 - accuracy: 1.0000 - val_loss: 0.1359 - val_accuracy: 0.9730
Epoch 10/40
19/19 [==============================] - 4s 198ms/step - loss: 2.2487e-04 - accuracy: 1.0000 - val_loss: 0.1375 - val_accuracy: 0.9730
Epoch 11/40
19/19 [==============================] - 4s 198ms/step - loss: 2.0621e-04 - accuracy: 1.0000 - val_loss: 0.1386 - val_accura

In [10]:
wandb.finish()

accuracy,▁███████████████████████████████████████
epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
loss,█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_accuracy,▁▄▅█████████████████████████████████████
val_loss,█▅▇▁▁▁▂▂▂▂▂▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▄▃▄▄▄▄▄▄▄▄▄▄▄
accuracy,1.0
best_epoch,4
best_val_loss,0.11806
epoch,39
loss,4e-05
val_accuracy,0.97297
